### Importing of libraries

In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sps

from sklearn.model_selection import train_test_split

### Dataset Loading

In [2]:
def load_data():
    return pd.read_csv("./data/data_train.csv",
                      sep=",",
                      names=["user_id","item_id","impl_rating"],
                      header=None,
                      skiprows=1,
                      dtype={"user_id": np.int32,
                             "item_id": np.int32,
                             "impl_rating": np.int32})


In [3]:
impl_ratings = load_data()

In [4]:
impl_ratings

,user_id,item_id,impl_rating
0,0,10080,1
1,0,19467,1
2,1,2665,1
3,1,7494,1
4,1,17068,1
...,...,...,...
113263,7945,2476,1
113264,7945,12319,1
113265,7945,21384,1
113266,7946,8699,1


### Data Preprocessing

In [5]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = impl_ratings.user_id.unique()
    unique_items = impl_ratings.item_id.unique()
    
    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()
    
    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)
    
    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})
    
    ratings = pd.merge(left=ratings,
                      right=mapping_user_id,
                      how="inner",
                      on="user_id")
    
    ratings = pd.merge(left=ratings,
                      right=mapping_item_id,
                      how="inner",
                      on="item_id")
    return ratings

In [6]:
ratings = preprocess_data(impl_ratings)

7947 0 7946
24896 0 25974


In [7]:
ratings

,user_id,item_id,impl_rating,mapped_user_id,mapped_item_id
0,0,10080,1,0,0
1,4342,10080,1,4342,0
2,5526,10080,1,5526,0
3,5923,10080,1,5923,0
4,0,19467,1,0,1
...,...,...,...,...,...
113263,7944,22542,1,7944,24891
113264,7944,24806,1,7944,24892
113265,7944,24912,1,7944,24893
113266,7944,24990,1,7944,24894


### Dataset Splitting

In [8]:
def dataset_splits(ratings,num_users,num_items, val_perc: float, test_perc: float):
    seed=9876
    
    (uid_training, uid_test,
    iid_training, iid_test,
    ratings_training, ratings_test) = train_test_split(ratings.mapped_user_id,
                                                      ratings.mapped_item_id,
                                                      ratings.impl_rating,
                                                      test_size=test_perc,
                                                      shuffle=True,
                                                      random_state=seed)
    (uid_training, uid_validation,
    iid_training, iid_validation,
    ratings_training, ratings_validation) = train_test_split(uid_training,
                                                            iid_training,
                                                            ratings_training,
                                                            test_size=val_perc)
    
    urm_train = sps.csr_matrix((ratings_training,(uid_training,iid_training)), shape=(num_users,num_items))
    urm_val = sps.csr_matrix((ratings_validation,(uid_validation,iid_validation)), shape=(num_users,num_items))
    urm_test = sps.csr_matrix((ratings_test,(uid_test,iid_test)), shape=(num_users,num_items))
    
    return urm_train,urm_val,urm_test

In [9]:
urm_train,urm_val,urm_test = dataset_splits(ratings, 
                                            num_users=7947, 
                                            num_items=24896, 
                                            val_perc=0.1, 
                                            test_perc=0.2)

In [10]:
urm_train

<7947x24896 sparse matrix of type '<class 'numpy.intc'>'
	with 81552 stored elements in Compressed Sparse Row format>

In [11]:
urm_val

<7947x24896 sparse matrix of type '<class 'numpy.intc'>'
	with 9062 stored elements in Compressed Sparse Row format>

In [12]:
urm_test

<7947x24896 sparse matrix of type '<class 'numpy.intc'>'
	with 22654 stored elements in Compressed Sparse Row format>

### First Function for similarity

In [13]:
def similarity(urm: sps.csc_matrix, shrink: int):
    item_weights = np.sqrt(
                        np.sum(
                        urm_train.tocsc().power(2),
                        axis=0
                        )
                   ).A.flatten()
    
    num_items = urm.shape[1]
    item_dot_product = urm.T.dot(urm).todense()
    
    weights = np.empty(shape=(num_items, num_items))
    for item_id in range(num_items):
        numerator = item_dot_product[item_id]
        denominator = item_weights[item_id]* item_weights + shrink + 1e-6 
        
        weights[item_id] = numerator / denominator
        
    np.fill_diagonal(weights, 0.0)
    
    return weights
    

In [14]:
%%time

weights = similarity(urm_train.tocsc(), shrink=5)

Wall time: 7.73 s


### Cosine Similarity

We can implement different versions of a cosine similarity. Some of these are faster and others are slower.

The most simple version is just to loop item by item and calculate the similarity of item pairs.
$$ W_{i,j} 
= cos(v_i, v_j) 
= \frac{v_i \cdot v_j}{|| v_i || ||v_j ||} 
= \frac{\Sigma_{u \in U}{URM_{u,i} \cdot URM_{u,j}}}{\sqrt{\Sigma_{u \in U}{URM_{u,i}^2}} \cdot \sqrt{\Sigma_{u \in U}{URM_{u,j}^2}} + shrink} $$


In [15]:
def naive_similarity(urm: sps.csc_matrix, shrink: int):
    num_items = urm.shape[1]
    weights = np.empty(shape=(num_items, num_items))
    for item_i in range(num_items):
        item_i_profile = urm[:, item_i] # mx1 vector
        
        for item_j in range(num_items):
            item_j_profile = urm[:, item_j] # mx1 vector
                      
            numerator = item_i_profile.T.dot(item_j_profile).todense()[0,0]
            denominator = (np.sqrt(np.sum(item_i_profile.power(2)))
                           * np.sqrt(np.sum(item_j_profile.power(2)))
                           + shrink
                           + 1e-6)
            
            weights[item_i, item_j] = numerator / denominator
    
    np.fill_diagonal(weights, 0.0)
    return weights


Another (faster) version of the similarity is by operating on vector products
$$ W_{i,I} 
= cos(v_i, URM_{I}) 
= \frac{v_i \cdot URM_{I}}{|| v_i || IW_{I} + shrink} $$

and where 

$$ IW_{i} = \sqrt{{\Sigma_{u \in U}{URM_{u,i}^2}}}$$

In [16]:
def vector_similarity(urm: sps.csc_matrix, shrink: int):
    item_weights = np.sqrt(
        np.sum(urm.power(2), axis=0)
    ).A.flatten()
    
    num_items = urm.shape[1]
    urm_t = urm.T
    weights = np.empty(shape=(num_items, num_items))
    for item_id in range(num_items):
        numerator = urm_t.dot(urm[:, item_id]).A.flatten()
        denominator = item_weights[item_id] * item_weights + shrink + 1e-6
        
        weights[item_id] = numerator / denominator
        
    np.fill_diagonal(weights, 0.0)
    return weights
    

Lastly, a faster but more memory-intensive version of the similarity is by operating on matrix products
$$ W  
= \frac{URM^{t} \cdot URM}{IW^{t} IW + shrink} $$

In [17]:
def matrix_similarity(urm: sps.csc_matrix, shrink: int):
    item_weights = np.sqrt(
        np.sum(urm.power(2), axis=0)
    ).A
    
    numerator = urm.T.dot(urm)
    denominator = item_weights.T.dot(item_weights) + shrink + 1e-6
    weights = numerator / denominator
    np.fill_diagonal(weights, 0.0)
    
    return weights

In [18]:
urm_csc = urm_train.tocsc()
shrink = 5
slice_size = 100

In [19]:
%%time 
naive_weights = naive_similarity(urm_csc[:slice_size,:slice_size], shrink)
naive_weights

Wall time: 7.8 s


array([[0.        , 0.16666664, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.16666664, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.16666664,
        0.16666664],
       [0.        , 0.        , 0.        , ..., 0.16666664, 0.        ,
        0.16666664],
       [0.        , 0.        , 0.        , ..., 0.16666664, 0.16666664,
        0.        ]])

In [20]:
%%time
vector_weights = vector_similarity(urm_csc[:slice_size,:slice_size], shrink)
vector_weights

Wall time: 35.9 ms


array([[0.        , 0.16666664, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.16666664, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.16666664,
        0.16666664],
       [0.        , 0.        , 0.        , ..., 0.16666664, 0.        ,
        0.16666664],
       [0.        , 0.        , 0.        , ..., 0.16666664, 0.16666664,
        0.        ]])

In [21]:
%%time
matrix_weights = matrix_similarity(urm_csc[:slice_size,:slice_size], shrink)
matrix_weights

Wall time: 18 ms


matrix([[0.        , 0.16666664, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.16666664, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.16666664,
         0.16666664],
        [0.        , 0.        , 0.        , ..., 0.16666664, 0.        ,
         0.16666664],
        [0.        , 0.        , 0.        , ..., 0.16666664, 0.16666664,
         0.        ]])

In [22]:
np.array_equal(naive_weights, vector_weights)

True

In [23]:
np.array_equal(vector_weights, matrix_weights)

True

### Collaborative Filtering ItemKNN Recommender

In [24]:
class CFItemKNN(object):
    
    def __init__(self, shrink:int):
        self.shrink = shrink
        self.weights = None
            
    
    def fit(self,urm_train: sps.csc_matrix, similarity_func: Callable[[sps.csc_matrix, int], np.array]):
        if not sps.isspmatrix_csc(urm_train):
            raise TypeError(f"We expected a CSC matrix, we got {type(urm_train)}")
        
        self.weights = similarity_func(urm_train, self.shrink)
        
    def recommend(self, user_id: int, urm_train: sps.csr_matrix, at: Optional[int] = None, remove_unseen: bool = True):
        user_profile = urm_train[user_id]
        
        ranking = user_profile.dot(self.weights).A.flatten()
        
        if remove_unseen:
            user_profile_start = urm_train.indptr[user_id]
            user_profile_end = urm_train.indptr[user_id+1]
            
            seen_items = urm_train.indices[user_profile_start:user_profile_end]
            
            ranking[seen_items] = -np.inf
            
        ranking = np.argsort(-ranking)
        return ranking[:at]


In [25]:
itemknn_recommender = CFItemKNN(shrink=50)

In [26]:
%%time

itemknn_recommender.fit(urm_train.tocsc(),matrix_similarity)

Wall time: 1min 17s


In [28]:
for user_id in range(10):
    print(itemknn_recommender.recommend(user_id=user_id,
                                  at=10, 
                                  urm_train=urm_train))

[ 1836 23196  1849  8104  3021  6319  2002  1707  9769  2896]
[1066 1704  133 1065 1526 1105 1460 2126 2627 2930]
[    0 16603 16602 16601 16600 16599 16598 16597 16596 16595]
[14995  2178  4589 23335 23336 10292 21660 21925 21924 15499]
[ 245 1642 1971 6855 2841 3115 1015 9825 3154 2280]
[ 1912  5434     3   329  1065   186  1460   644 13390  2800]
[ 6294  1609  3476  1606  3498  3420  1424  1395 12479  3470]
[   54  1369  9473 11641 11644  2789  7405  1996  1788   354]
[3041 6268 2083  303 1827  551  644  304  186 1492]
[3153 5105 1718 9815 2026 1066 2122 2134  489  445]


### Evaluation Metrics

In [29]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

### Evaluator

In [38]:
def evaluator(recommender: object, urm_train: sps.csr_matrix, urm_test: sps.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]
        
        relevant_items = urm_test.indices[user_profile_start:user_profile_end]
        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        recommendations = recommender.recommend(user_id=user_id,
                                               at=recommendation_length,
                                               urm_train=urm_train)
        
        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)
        
        num_users_evaluated += 1
        
    accum_precision /= max(num_users_evaluated,1)
    accum_recall /= max(num_users_evaluated,1) 
    accum_map /= max(num_users_evaluated,1)
        
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [41]:
%%time
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemknn_recommender,
                                                                                            urm_train,
                                                                                            urm_test)

Wall time: 16.7 s


In [42]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

(0.027064712191633433, 0.08781670101760261, 0.04267728300883501, 5594, 2353)

### Hyperparameter Tuning

In [45]:
def hyperparameter_tuning():
    shrinks = [0,1,5,10,50]
    results = []
    for shrink in shrinks:
        print(f"Currently trying shrink {shrink}")
        itemknn_recommender = CFItemKNN(shrink=shrink)
        itemknn_recommender.fit(urm_train.tocsc(), matrix_similarity)
        
        ev_precision, ev_recall, ev_map,_,_ = evaluator(itemknn_recommender, urm_train, urm_val)
        
        results.append((shrink,(ev_precision, ev_recall, ev_map)))
        
    return results

In [46]:
%%time

hyperparameter_results = hyperparameter_tuning()

Currently trying shrink 0
Currently trying shrink 1
Currently trying shrink 5
Currently trying shrink 10
Currently trying shrink 50
Wall time: 6min 44s


In [47]:
hyperparameter_results

[(0, (0.009231619679380916, 0.04789152139130158, 0.01960429709115185)),
 (1, (0.011138750690989562, 0.05763867150723729, 0.02285195168756034)),
 (5, (0.014980652294085246, 0.07551376594926669, 0.03134376992100608)),
 (10, (0.016362631288004553, 0.08084851985232039, 0.03512755976876345)),
 (50, (0.016832504145937108, 0.08456855483114999, 0.036860233864341986))]

### Submission to competition

In [51]:
best_shrink = 50
urm_train_validation = urm_train + urm_val

In [52]:
best_recommender = CFItemKNN(shrink=best_shrink)
best_recommender.fit(urm_train_validation.tocsc(), matrix_similarity)

### User to recommend

In [74]:
users_to_recommend = pd.read_csv("./data/data_target_users_test.csv",
                      names=["user_id"],
                      header=None,
                      skiprows=1,
                      dtype={"user_id": np.int32})
users_to_recommend

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [85]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix, recommender: object):
    user_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates().sort_values(by=['user_id'])
    
    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    
    recommendation_length = 10
    submission = []
    for idx, row in user_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        
        recommendations = recommender.recommend(user_id= mapped_user_id,
                                                urm_train=urm_train,
                                                at=recommendation_length)
        
        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))
    
    return submission

In [86]:
submission = prepare_submission(ratings, users_to_recommend.user_id, urm_train_validation, best_recommender)

In [ ]:
submission

In [95]:
def write_submission(submission, namefile: str):
    with open("./submissions/"+namefile+".csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submission:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

In [96]:
from datetime import date
today = date.today().strftime("%d-%m-%y")
write_submission(submission, today)